# 데이터 수집

### 데이터 크롤링 

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article

def make_urllist(page_num, code, date): 
    urllist= []
    for i in range(1, page_num + 1):
        url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
        news = requests.get(url)

        # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
        soup = BeautifulSoup(news.content, 'html.parser')

        # CASE 1
        news_list = soup.select('.newsflash_body .type06_headline li dl')
        # CASE 2
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
        # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
        for line in news_list:
            urllist.append(line.a.get('href'))
        return urllist

url_list = make_urllist(2, 101, 20200506)
print('뉴스 기사의 개수: ',len(url_list))

idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.text)

    #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
    df = pd.DataFrame({'news': text_list})

    #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
    df['code'] = idx2word[str(code)]
    return df

data = make_data(url_list, 101)
#- 상위 10개만 출력해봅니다.
data[:10]

code_list = [101, 102, 103, 105]

code_list

def make_total_data(page_num, code_list, date):
    df = None

    for code in code_list:
        url_list = make_urllist(page_num, code, date)
        df_temp = make_data(url_list, code)
        print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp

    return df

df = make_total_data(1, code_list, 20200506)

print('뉴스 기사의 개수: ',len(df))

df.sample(10)


import os

# 데이터프레임 파일을 csv 파일로 저장합니다.
# 저장경로는 이번 프로젝트를 위해 만든 폴더로 지정해 주세요.
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df.to_csv(csv_path, index=False)

if os.path.exists(csv_path):
    print('{} File Saved!'.format(csv_path))


뉴스 기사의 개수:  20
101번 코드에 대한 데이터를 만들었습니다.
102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.
뉴스 기사의 개수:  80
/home/aiffel0042/aiffel/news_crawler/news_data.csv File Saved!


---
### 데이터 전처리 및 세팅

In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from konlpy.tag import Mecab, Hannanum, Kkma, Komoran, Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

csv_path = os.getenv("HOME") + "/project/aiffel-lms/F3_Crawler/news_data.csv"
csv_path2 = os.getenv("HOME") + "/project/aiffel-lms/F3_Crawler/news_data2.csv"
stop_path = os.getenv("HOME") + "/project/aiffel-lms/F3_Crawler/kor_stopwords.txt"
f = open(stop_path, 'r')
lines = f.readlines()

stopwords = []
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()

print(stopwords)

df1 = pd.read_table(csv_path, sep=',')
df2 = pd.read_table(csv_path2, sep=',')

df = pd.concat([df1, df2])

df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

df.drop_duplicates(subset=['news'], inplace=True)


mecab = Mecab()
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

def preprocessing(data, tokenaizer):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = tokenaizer.morphs(sentence) 
        #- 불용어 제거
        #temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

mecab_data = preprocessing(df['news'], mecab)
#han_data = preprocessing(df['news'], hannanum)
#okt_data = preprocessing(df['news'], okt)
#kkma_data = preprocessing(df['news'], kkma)
#komoran_data = preprocessing(df['news'], komoran)

X_train, X_test, y_train, y_test = train_test_split(mecab_data, df['code'], random_state = 19)

print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)



['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤\t', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개\t', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓\t']
훈련용 뉴스 기사의 개수 : 4595
테스트용 뉴스 기사의 개수 :  1532
훈련용 레이블의 개수 :  4595
테스트용 레이블의 개수 :  1532


---
# 머신러닝

In [9]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)

def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.89      0.72      0.79       274
          경제       0.94      0.45      0.60       229
          사회       0.71      0.94      0.81       552
       생활/문화       0.79      0.78      0.79       477

    accuracy                           0.78      1532
   macro avg       0.83      0.72      0.75      1532
weighted avg       0.80      0.78      0.77      1532



---
# 리포트

1. 약 2500개의 데이터 샘플로 진행했을 때, IT/과학 분야의 기사 데이터가 부족해 정확도가 제대로 나오지 않음

2. 약 3500개의 데이터를 더 추가해 진행하자 정확도가 획기적으로 개선됨

3. 불용어 리스트를 임의로 지정한 리스트가 아니라 공개된 공용 불용어 파일을 받아 리스트로 불러옴

4. 분석기마다 정확도의 차이는 크지 않음 (0.1미만차이)

5. 분석기마다 속도의 차이는 매우 큼(mecab이 가장 빠르고 kkma가 가장 느림)

6. 따라서 mecab 분석기를 선택해 진행하였음